# INTRODUCTION
This notebook outlines the following steps for downloading basic data:
- 1x1 km grids are created for the Basque Country.

- Data with emissions and geolocation information is downloaded directly from Google Earth.

- The database will begin in June 2018, the first month for which Sentinel-5P data is available.

- Data from the OMI/AURA Project is downloaded for Sentinel-5P level backprojection. The goal is to establish a data series starting from October 2017.

- The first version of the data is made available (without validation or imputation, only original data).

### Preamble
Installing and importing libraries.

In [1]:
#!pip install geopandas
#!pip install earthengine-api
#!pip install geemap
#!pip install relativedelta
#!pip install openpyxl

In [2]:
import geopandas as gpd
from shapely.geometry import box
import numpy as np
import zipfile
import os
import requests
import pandas as pd

import geemap
from datetime import datetime
from dateutil.relativedelta import relativedelta

We establish the working path, leaving the option for execution in **colab** or **local**.

In [3]:
import os
import sys

# If we're in Google Colab
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # Path inside Colab
    BASE_DIR = "/content/data/"
    os.makedirs(BASE_DIR, exist_ok=True)

else:
    # Local path based on the notebook's location
    NOTEBOOK_DIR = os.getcwd() 
    PROJECT_DIR = os.path.dirname(NOTEBOOK_DIR)
    BASE_DIR = os.path.join(PROJECT_DIR, "data")  # project/data/

    os.makedirs(BASE_DIR, exist_ok=True)

In [35]:
# Load utils
import importlib
project_root = os.path.abspath("..")
sys.path.append(project_root)

from src.utils import *
import src.utils as u
importlib.reload(u)

<module 'src.utils' from 'C:\\Users\\camil\\Desktop\\Geospatial_Approach\\src\\utils.py'>

## Data Download

We will download the map of the Basque Country from the GADM Global Database. This is a geographic resource that provides administrative boundaries worldwide for countries and their subdivisions.

### 1. Download map and create grids

In [5]:
# 1. Download the ZIP file from GADM
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_ESP_shp.zip"
zip_path = os.path.join(BASE_DIR,"gadm41_ESP_shp.zip")

if not os.path.exists(zip_path):
    print("Downloading...")
    r = requests.get(url)
    with open(zip_path, "wb") as f:
        f.write(r.content)
else:
    print("ZIP already downloaded.")


ZIP already downloaded.


In [6]:
# 2. Extract the ZIP file
extract_dir = os.path.join(BASE_DIR, "gadm41_ESP_shp")

# Create folder if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

#### Loading the Shapefile

A shapefile is a vector geospatial data format widely used in Geographic Information Systems (GIS). Esri developed it to store the location, shape, and attributes of geographic features. A shapefile is not a single file, but a set of related files that work together to define a geographic layer.

Main components of a shapefile:

- .shp: Contains the geometry of the features (points, lines, polygons).

- .shx: An index file that allows for quick searches within the .shp file.

- .dbf: A dBASE table that stores the attributes (descriptive information) of each feature. 

In [7]:
# 3. Load shapefile and filter Basque Country
shp_path = os.path.join(BASE_DIR,"gadm41_ESP_shp/gadm41_ESP_1.shp")
gdf = gpd.read_file(shp_path)
pais_vasco = gdf[gdf["NAME_1"].str.lower().str.contains("vasco|basque")].copy()

In [8]:
# 4. Reproject to metric coordinates (to work with 1km cells)
pais_vasco = pais_vasco.to_crs(epsg=25830)  # UTM Zone 30N

In [9]:
# 5. Save
output_path = os.path.join(BASE_DIR, "pais_vasco.shp")

pais_vasco.to_file(output_path)

In [10]:
# Verify
pais_vasco

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
15,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,"MULTIPOLYGON (((528223.332 4705799.236, 527460..."


In [11]:
# We use the GeoDataFrame `pais_vasco` which is already loaded (EPSG:25830)

# Extract limits
minx, miny, maxx, maxy = pais_vasco.total_bounds
cell_size = 1000  # 1 km en meters

# Create cell grid
grid_cells = []
for x in np.arange(minx, maxx, cell_size):
    for y in np.arange(miny, maxy, cell_size):
        cell = box(x, y, x + cell_size, y + cell_size)
        grid_cells.append(cell)

# Create GeoDataFrame with the complete grid
grid = gpd.GeoDataFrame(geometry=grid_cells, crs=pais_vasco.crs)

# Cut out only cells that intersect with the Basque Country
grid_clip = gpd.overlay(grid, pais_vasco, how='intersection')
grid_clip["grid_id"] = range(len(grid_clip))
# Save
output_path = os.path.join(BASE_DIR, "grilla_1km_pais_vasco.geojson")
grid_clip.to_file(output_path, driver="GeoJSON")

print(f"{len(grid_clip)} of 1x1 km were created.")

7718 of 1x1 km were created.


In [12]:
# Load original grid
grid = gpd.read_file(output_path)

# export grid_id + geometry
output_path2= os.path.join(BASE_DIR, "grid_id_geometria.gpkg")
grid[['grid_id', 'geometry']].to_file(output_path2, driver="GPKG")

In [13]:
# create centroids
grid_centroids = grid.copy()
grid_centroids["geometry"] = grid_centroids.centroid

# Extract lat/lon for CSV type table
grid_centroids = grid_centroids.to_crs(epsg=4326)
grid_centroids["lon"] = grid_centroids.geometry.x
grid_centroids["lat"] = grid_centroids.geometry.y

# Export as CSV
output_path3= os.path.join(BASE_DIR, "grid_id_centroides.csv")
grid_centroids[["grid_id", "lat", "lon"]].to_csv(output_path3, index=False)

### 2. Downloading Emissions COPERNICUS GOOGLE EARTH

#### Connecting to Google Earth
We will download the emissions from this platform. It is necessary to have previously initialized a project in Google Earth.

For this data download, we will connect to the following projects: **COPERNICUS/S5P/OFFL/.**

In [14]:
# Autentification in GoogleEarth
import ee
ee.Authenticate()
ee.Initialize()

In [15]:
# Initialize Earth Engine
ee.Initialize()

# Load the grid
grid = gpd.read_file(output_path)
points = grid.copy()
points["geometry"] = points["geometry"].centroid
points = points.to_crs(epsg=4326)

fc = geodf_to_fc(points)

### Explanation: Converting from a native 3.5 x 7 km pixel to our 1 x 1 km grids

Not all 1x1 km cells that "fall" within a native 3.5x7 km pixel will necessarily receive the exact same average emission value (e.g., CO). However, the information received by those 1x1 km cells is derived directly from that larger pixel (and adjacent pixels if there is overlap).

**How ​​reduceRegions works with different resolutions**: The code uses *co.reduceRegions(collection=fc_chunk, reducer=ee.Reducer.mean(), scale=1000)*. Let's analyze each part:

- *co = collection.mean()*: First, for each month, GEE creates a monthly average image from all the daily SO2 data. The pixels in this average image maintain their original native resolution of 3.5 x 7 km.

- *fc_chunk*: This is our collection of geometries, where each geometry is a polygon representing a 1 km x 1 km cell.

- *reducer=ee.Reducer.mean()*: For each 1 km x 1 km polygon in our fc_chunk, GEE will perform the following calculation:

- It will identify all pixels in the monthly average SO2 image (which are 3.5 x 7 km in size) that intersect that specific 1 km x 1 km polygon.

- It will calculate the average SO2 value of those intersected pixels.

- If a 1 x 1 km pixel is completely contained within a single 3.5 x 7 km pixel, and that 3.5 x 7 km pixel does not overlap with any other 1 x 1 km cell, then the 1 x 1 km cell will take the value of that larger pixel.

But here's the key: A 3.5x7 km pixel is much larger than a 1x1 km pixel (approximately 24.5 km² vs. 1 km²). Therefore, it's very common for a single 3.5x7 km pixel to overlap with multiple 1x1 km cells.

It's also common for a 1x1 km cell to lie on the edge of two or more 3.5x7 km pixels. In this case, the value of the 1x1 km cell will be a weighted average of the values ​​of the 3.5x7 km pixels that intersect it.

- scale=1000: This parameter tells GEE to perform the average calculation at a resolution of 1000 meters (1 km). This means that GEE will internally project or resample the source pixels (3.5x7 km) to this 1 km scale before performing the zonal calculation for your polygons. Although the original data comes from large pixels, the calculation is done considering our 1 km scale.

**In summary:**

It's not that all 1x1 km cells within a single 3.5x7 km cell receive an identical value if that 3.5x7 km cell also overlaps with other 1x1 km cells that cross its boundary.

Rather, each 1x1 km cell receives an average value that is calculated specifically for its geometry, based on the values ​​of the 3.5x7 km pixels that cover it (or that are resampled to 1 km for the calculation).

If several 1x1 km cells are very close together and fall predominantly on the same 3.5x7 km pixel, their SO2 values ​​will naturally be very similar or even identical if there is no influence from adjacent 3.5x7 km pixels.

What you're doing is a valid and standard way to "downscale" or "disaggregate" the coarse resolution information to your finer grid, obtaining a representative emission value for each of your 1km x 1km quadrants. The machine learning model will learn from these relationships.

#### Descarga de NO2

Además de la descarga, creamos la base df con el *grid_id*, *year*, *month* y *NO2* para la serie 06-2019 a 12-2022.

- Proyecto: COPERNICUS/S5P/OFFL/L3_NO2
- Variable escogida: "NO2_column_number_density"
- Disponibilidad: 2018-06-28T10:24:07Z–2025-07-05T23:04:25Z

In [33]:
# We download NO2 emissions
df_no2 = u.extract_full_timeseries(
    dataset="COPERNICUS/S5P/OFFL/L3_NO2",
    variable="NO2_column_number_density",
    points=points,
    start_year=2018,
    end_year=2022
)

# Save .csv with monthly NO2
output_path_e= os.path.join(BASE_DIR, "NO2_mensual_por_grid.csv")
df_no2.to_csv(output_path_e, index=False)


🔵 Iniciando extracción: COPERNICUS/S5P/OFFL/L3_NO2 (NO2_column_number_density)
  → 2018-01
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_NO2 2018-01
  → 2018-02
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_NO2 2018-02
  → 2018-03
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_NO2 2018-03
  → 2018-04
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_NO2 2018-04
  → 2018-05
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_NO2 2018-05
  → 2018-06
  → 2018-07
  → 2018-08
  → 2018-09
  → 2018-10
  → 2018-11
  → 2018-12
  → 2019-01
  → 2019-02
  → 2019-03
  → 2019-04
  → 2019-05
  → 2019-06
  → 2019-07
  → 2019-08
  → 2019-09
  → 2019-10
  → 2019-11
  → 2019-12
  → 2020-01
  → 2020-02
  → 2020-03
  → 2020-04
  → 2020-05
  → 2020-06
  → 2020-07
  → 2020-08
  → 2020-09
  → 2020-10
  → 2020-11
  → 2020-12
  → 2021-01
  → 2021-02
  → 2021-03
  → 2021-04
  → 2021-05
  → 2021-06
  → 2021-07
  → 2021-08
  → 2021-09
  → 2021-10
  → 2021-11
  → 2021-12
  → 2022-01
  → 2022-02
  → 2022-03
  → 2022-04
  → 2022-05
  → 2022-

In [34]:
# Save .csv with monthly NO2
output_path_e= os.path.join(BASE_DIR, "NO2_mensual_por_grid.csv")
df_no2.to_csv(output_path_e, index=False)

#### Descarga de CO

- Proyecto: COPERNICUS/S5P/OFFL/L3_CO
- Variable escogida: "CO_column_number_density"
- Disponibilidad: 22018-06-28T10:24:07Z–2025-07-12T22:32:34Z

In [36]:
# We download CO emissions
df_co = u.extract_full_timeseries(
    dataset="COPERNICUS/S5P/OFFL/L3_CO",
    variable="CO_column_number_density",
    points=points,
    start_year=2018,
    end_year=2022
)

# Save .csv with monthly CO
output_path_e= os.path.join(BASE_DIR, "CO_mensual_por_grid.csv")
df_co.to_csv(output_path_e, index=False)


🔵 Iniciando extracción: COPERNICUS/S5P/OFFL/L3_CO (CO_column_number_density)
  → 2018-01
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_CO 2018-01
  → 2018-02
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_CO 2018-02
  → 2018-03
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_CO 2018-03
  → 2018-04
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_CO 2018-04
  → 2018-05
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_CO 2018-05
  → 2018-06
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_CO 2018-06
  → 2018-07
  → 2018-08
  → 2018-09
  → 2018-10
  → 2018-11
  → 2018-12
  → 2019-01
  → 2019-02
  → 2019-03
  → 2019-04
  → 2019-05
  → 2019-06
  → 2019-07
  → 2019-08
  → 2019-09
  → 2019-10
  → 2019-11
  → 2019-12
  → 2020-01
  → 2020-02
  → 2020-03
  → 2020-04
  → 2020-05
  → 2020-06
  → 2020-07
  → 2020-08
  → 2020-09
  → 2020-10
  → 2020-11
  → 2020-12
  → 2021-01
  → 2021-02
  → 2021-03
  → 2021-04
  → 2021-05
  → 2021-06
  → 2021-07
  → 2021-08
  → 2021-09
  → 2021-10
  → 2021-11
  → 2021-12
  → 2022-01
  → 2022-0

In [37]:
# We gathered the data in df
df = df_no2
df = df.merge(df_co, on=["grid_id", "year", "month"], how="left")

In [39]:
# Save df
output_path = os.path.join(BASE_DIR, "bd_consolidado.csv")
df.to_csv(output_path, index=False)

#### Descarga de SO2

- Proyecto: COPERNICUS/S5P/OFFL/L3_SO2
- Variable escogida: "SO2_column_number_density"
- Disponibilidad: 2018-07-10T11:17:44Z–2025-07-14T14:01:39Z

En la versión 1, tenía meses a nivel de grid con nivel negativo. Esto no tiene sentido, por lo que se incorporó que ates de agrupar, solo considere *SO2_column_number_density >=0*.

In [40]:
# We download SO2 emissions
df_so2 = u.extract_full_timeseries(
    dataset="COPERNICUS/S5P/OFFL/L3_SO2",
    variable="SO2_column_number_density",
    points=points,
    start_year=2018,
    end_year=2022
)

# Save .csv with monthly SO2
output_path_e= os.path.join(BASE_DIR, "SO2_mensual_por_grid.csv")
df_so2.to_csv(output_path_e, index=False)


🔵 Iniciando extracción: COPERNICUS/S5P/OFFL/L3_SO2 (SO2_column_number_density)
  → 2018-01
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-01
  → 2018-02
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-02
  → 2018-03
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-03
  → 2018-04
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-04
  → 2018-05
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-05
  → 2018-06
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-06
  → 2018-07
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-07
  → 2018-08
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-08
  → 2018-09
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-09
  → 2018-10
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_SO2 2018-10
  → 2018-11
  → 2018-12
  → 2019-01
  → 2019-02
  → 2019-03
  → 2019-04
  → 2019-05
  → 2019-06
  → 2019-07
  → 2019-08
  → 2019-09
  → 2019-10
  → 2019-11
  → 2019-12
  → 2020-01
  → 2020-02
  → 2020-03
  → 2020-04
  → 2020-05
  → 2020-06
  → 2020-0

In [41]:
df_so2 = df_so2.groupby(["grid_id", "year", "month"], as_index=False).mean()

In [43]:
# We gathered the data in df
df = df.merge(df_so2, on=["grid_id", "year", "month"], how="left")

In [45]:
# Save df
output_path = os.path.join(BASE_DIR, "bd_consolidado.csv")
df.to_csv(output_path, index=False)

#### Download O3

- Proyecto: COPERNICUS/S5P/NRTI/L3_O3
- Variable escogida: "O3_column_number_density"
- Disponibilidad: 2018-07-10T11:02:44Z–2025-07-14T14:01:39Z

In [47]:
# We download O3 emissions
df_o3 = u.extract_full_timeseries(
    dataset="COPERNICUS/S5P/NRTI/L3_O3",
    variable="O3_column_number_density",
    points=points,
    start_year=2018,
    end_year=2022
)

# Save .csv with monthly O3
output_path_e= os.path.join(BASE_DIR, "O3_mensual_por_grid.csv")
df_o3.to_csv(output_path_e, index=False)       


🔵 Iniciando extracción: COPERNICUS/S5P/NRTI/L3_O3 (O3_column_number_density)
  → 2018-01
⚠ No hay datos para COPERNICUS/S5P/NRTI/L3_O3 2018-01
  → 2018-02
⚠ No hay datos para COPERNICUS/S5P/NRTI/L3_O3 2018-02
  → 2018-03
⚠ No hay datos para COPERNICUS/S5P/NRTI/L3_O3 2018-03
  → 2018-04
⚠ No hay datos para COPERNICUS/S5P/NRTI/L3_O3 2018-04
  → 2018-05
⚠ No hay datos para COPERNICUS/S5P/NRTI/L3_O3 2018-05
  → 2018-06
⚠ No hay datos para COPERNICUS/S5P/NRTI/L3_O3 2018-06
  → 2018-07
  → 2018-08
  → 2018-09
  → 2018-10
  → 2018-11
  → 2018-12
  → 2019-01
  → 2019-02
  → 2019-03
  → 2019-04
  → 2019-05
  → 2019-06
  → 2019-07
  → 2019-08
  → 2019-09
  → 2019-10
  → 2019-11
  → 2019-12
  → 2020-01
  → 2020-02
  → 2020-03
  → 2020-04
  → 2020-05
  → 2020-06
  → 2020-07
  → 2020-08
  → 2020-09
  → 2020-10
  → 2020-11
  → 2020-12
  → 2021-01
  → 2021-02
  → 2021-03
  → 2021-04
  → 2021-05
  → 2021-06
  → 2021-07
  → 2021-08
  → 2021-09
  → 2021-10
  → 2021-11
  → 2021-12
  → 2022-01
  → 2022-0

In [48]:
# We gathered the data in df
df = df.merge(df_o3, on=["grid_id", "year", "month"], how="left")

In [50]:
# Save df
output_path = os.path.join(BASE_DIR, "bd_consolidado.csv")
df.to_csv(output_path, index=False)

#### Downloading AAI

The **absorbing_aerosol_index (AAI)** is a direct measure of the prevalence of UV-absorbing aerosols, such as desert dust, smoke (from biomass fires), and volcanic ash. For the study of steelmaking activity, the AAI is highly relevant because:

It indicates combustion/industrial aerosols: Combustion processes in steelmaking can generate soot particles and other aerosols that absorb UV radiation. A positive AAI value indicates the presence of these types of aerosols.

It is unaffected by clouds: Unlike other aerosol measurements (such as standard aerosol optical depth) that can be obstructed by clouds, the AAI can be calculated even in their presence. This ensures greater coverage and data availability, which is advantageous for your time series.  

- Proyecto: COPERNICUS/S5P/OFFL/L3_AER_AI
- Variable escogida: "absorbing_aerosol_index"
- Disponibilidad: 2018-07-04T13:34:21Z–2025-07-13T00:14:03Z

In [ ]:
# We download AAI emissions
df_aai = u.extract_full_timeseries(
    dataset="COPERNICUS/S5P/OFFL/L3_AER_AI",
    variable="absorbing_aerosol_index",
    points=points,
    start_year=2018,
    end_year=2022
)

# Save .csv with monthly AAI
output_path_e= os.path.join(BASE_DIR, "AAI_mensual_por_grid.csv")
df_aai.to_csv(output_path_e, index=False)  


🔵 Iniciando extracción: COPERNICUS/S5P/OFFL/L3_AER_AI (absorbing_aerosol_index)
  → 2018-01
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_AER_AI 2018-01
  → 2018-02
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_AER_AI 2018-02
  → 2018-03
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_AER_AI 2018-03
  → 2018-04
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_AER_AI 2018-04
  → 2018-05
⚠ No hay datos para COPERNICUS/S5P/OFFL/L3_AER_AI 2018-05
  → 2018-06
  → 2018-07
  → 2018-08
  → 2018-09
  → 2018-10
  → 2018-11
  → 2018-12
  → 2019-01
  → 2019-02
  → 2019-03
  → 2019-04
  → 2019-05
  → 2019-06
  → 2019-07
  → 2019-08
  → 2019-09
  → 2019-10
  → 2019-11
  → 2019-12
  → 2020-01
  → 2020-02
  → 2020-03
  → 2020-04
  → 2020-05
  → 2020-06
  → 2020-07
  → 2020-08
  → 2020-09
  → 2020-10
  → 2020-11
  → 2020-12
  → 2021-01
  → 2021-02
  → 2021-03


In [ ]:
df = df.merge(df_aai, on=["grid_id", "year", "month"], how="left")

In [ ]:
# Save df
output_path = os.path.join(BASE_DIR, "bd_consolidado.csv")
df.to_csv(output_path, index=False)

### Descarga de NTL
Descargamos indicador de luz nocturna.  
Usamos la colección: NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG. Esta tiene dos campos:  
- **avg_rad**: Esta es la que utilizamos.
- **cf_cvg**: Coberturas sin nubes: La cantidad total de observaciones que se ingresaron en cada píxel. Esta banda se puede usar para identificar áreas con poca cantidad de observaciones en las que se reduce la calidad. Usar esta para imputar y validar lo que estamos usando.

In [ ]:
# We download NTL emissions
df_ntl = u.extract_full_timeseries(
    dataset="NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG",
    variable="avg_rad",
    points=points,
    start_year=2017,
    end_year=2022
)

# Save .csv with monthly NTL
output_path_e= os.path.join(BASE_DIR, "NTL_mensual_por_grid.csv")
df_ntl.to_csv(output_path_e, index=False)  

In [ ]:
df = df.merge(df_ntl, on=["grid_id", "year", "month"], how="right")

In [ ]:
#Guardamos el consolidado
df = df[['grid_id', 'year', 'month', 'NO2', 'CO','SO2','O3', 'AAI', 'NTL']]
df.to_csv("bd_consolidado.csv", index=False)

### Descarga de LST
Land Surface Temperature. Las colecciones más comunes y adecuadas para LST provienen de los sensores MODIS (Moderate Resolution Imaging Spectroradiometer) a bordo de los satélites Aqua y Terra.  
Utilizamos la colección: **MODIS/061/MOD11A2**. Esta contiene los productos de 8 días de promedio, que ya han sido corregidos y compuestos para reducir el efecto de las nubes y otras interferencias.  
El campo utilizado es: **LST_Day_1km** que contiene Temperatura diurna de la superficie terrestre.



In [ ]:
# We download LST emissions
df_lst = u.extract_full_timeseries(
    dataset="MODIS/061/MOD11A2",
    variable="LST_Day_1km",
    points=points,
    start_year=2017,
    end_year=2022
)

# Save .csv with monthly LST
output_path_e= os.path.join(BASE_DIR, "LST_mensual_por_grid.csv")
df_lst.to_csv(output_path_e, index=False) 

In [ ]:
df = df.merge(df_lst, on=["grid_id", "year", "month"], how="left")

In [ ]:
#Guardamos el consolidado
df = df[['grid_id', 'year', 'month', 'NO2', 'CO','SO2','O3', 'AAI', 'NTL','LST']]
df.to_csv("bd_consolidado.csv", index=False)

### Descargamos PM2.5
- Proyecto: "ECMWF/CAMS/NRT"
- Campo: particulate_matter_d_less_than_25_um_surface

In [ ]:
# We download PM emissions
df_pm = u.extract_full_timeseries(
    dataset="ECMWF/CAMS/NRT",
    variable="particulate_matter_d_less_than_25_um_surface",
    points=points,
    start_year=2017,
    end_year=2022
)

# Save .csv with monthly PM
output_path_e= os.path.join(BASE_DIR, "PM_mensual_por_grid.csv")
df_pm.to_csv(output_path_e, index=False) 

Procesando 2018-01
Procesando 2018-02
Procesando 2018-03
Procesando 2018-04
Procesando 2018-05
Procesando 2018-06
Procesando 2018-07
Procesando 2018-08
Procesando 2018-09
Procesando 2018-10
Procesando 2018-11
Procesando 2018-12
Procesando 2019-01
Procesando 2019-02
Procesando 2019-03
Procesando 2019-04
Procesando 2019-05
Procesando 2019-06
Procesando 2019-07
Procesando 2019-08
Procesando 2019-09
Procesando 2019-10
Procesando 2019-11
Procesando 2019-12
Procesando 2020-01
Procesando 2020-02
Procesando 2020-03
Procesando 2020-04
Procesando 2020-05
Procesando 2020-06
Procesando 2020-07
Procesando 2020-08
Procesando 2020-09
Procesando 2020-10
Procesando 2020-11
Procesando 2020-12
Procesando 2021-01
Procesando 2021-02
Procesando 2021-03
Procesando 2021-04
Procesando 2021-05
Procesando 2021-06
Procesando 2021-07
Procesando 2021-08
Procesando 2021-09
Procesando 2021-10
Procesando 2021-11
Procesando 2021-12
Procesando 2022-01
Procesando 2022-02
Procesando 2022-03
Procesando 2022-04
Procesando 2

In [ ]:
df = df.merge(df_pm, on=["grid_id", "year", "month"], how="left")

In [ ]:
#Guardamos el consolidado
df = df[['grid_id', 'year', 'month', 'NO2', 'CO','SO2','O3', 'AAI', 'NTL','LST','PM25']]
df.to_csv("bd_consolidado.csv", index=False)

## Revisión de series
En este apartado haremos estadística descriptiva de la serie mensual descargada.
  
  
Comenzamos analizando los resultados agrupados a nivel de País Vasco. Para esto sumaremos los promedios mensuales por grid. Esto tal vez no tenga tanto sentido de interpretación, pero es útil para ver meses sin emisión y la dinámica en la serie.

In [ ]:
# Cargamos el df es csv ya guardado
df = pd.read_csv("bd_consolidado.csv")

In [ ]:
df_grouped = df.groupby(['year', 'month'])[['NO2', 'SO2', 'O3', 'CO', 'AAI', 'NTL','LST']].mean().reset_index()

In [ ]:
import matplotlib.pyplot as plt

df_grouped['date'] = pd.to_datetime(df_grouped[['year', 'month']].assign(day=1))
df_grouped = df_grouped.sort_values('date')

In [ ]:
# Graficamos
# Lista de contaminantes
import matplotlib.dates as mdates
contaminantes = ['NO2', 'SO2', 'O3', 'CO', 'AAI', 'NTL', 'LST']

fig, axs = plt.subplots(len(contaminantes), 1, figsize=(15, 12), sharex=True)

for i, cont in enumerate(contaminantes):
    axs[i].bar(df_grouped['date'], df_grouped[cont], width=20)
    axs[i].set_ylabel(cont)
    axs[i].set_title(f'Concentración mensual de {cont}')
    axs[i].grid(True)

# Formatear eje x para mostrar año y mes
axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
axs[-1].xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # una etiqueta cada 3 meses
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

## Creación de campo "tiene planta"
Ahora tenemos que marcar los grid que tienen plantas en nuestro df.  
  
Comenzamos importando y luego pasando el data frame de plantas a tipo GeoDataFrame

In [ ]:
# Importamos las plantas
df_plantas = pd.read_excel("./datos_empresa/produccion empresas vascas 18-22.xlsx", sheet_name="bd_plantas")

In [ ]:
from shapely.geometry import Point

df_plantas["geometry"] = df_plantas.apply(lambda row: Point(row["Longitud"], row["Latitud"]), axis=1)
gdf_plantas = gpd.GeoDataFrame(df_plantas, geometry="geometry", crs="EPSG:4326")

# Reproyectar grid al mismo CRS si es necesario
if grid.crs != gdf_plantas.crs:
    grid = grid.to_crs(gdf_plantas.crs)

In [ ]:
# Se une cada planta con el grid en el que cae
plantas_con_grid = gpd.sjoin(gdf_plantas, grid[["grid_id", "geometry"]], how="left", predicate="within")
plantas_con_grid

,EMPRESA,Latitud,Longitud,localidad,provincia,geometry,index_right,grid_id
0,1,43.034900,-3.001310,Amurrio,Alava,POINT (-3.00131 43.0349),1235,1235
1,2,43.295011,-2.910000,Loiu,Vizcaya,POINT (-2.91 43.29501),1825,1825
2,3,43.041698,-2.216001,Olaberría,Guipuzcoa,POINT (-2.216 43.0417),6618,6618
3,4,43.310400,-2.998450,Sestao,Vizcaya,POINT (-2.99845 43.3104),1265,1265
4,5,43.309900,-3.020770,Valle de Trapaga,Vizcaya,POINT (-3.02077 43.3099),1119,1119
5,6,43.299400,-3.006360,Sestao,Vizcaya,POINT (-3.00636 43.2994),1190,1190
6,7,43.268700,-3.153110,Basauri,Vizcaya,POINT (-3.15311 43.2687),515,515
7,8,43.077600,-2.995680,Amurrio,Alava,POINT (-2.99568 43.0776),1240,1240


In [ ]:
grid.to_file("grilla_pais_vasco.gpkg", driver="GPKG")

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,grid_id,geometry
0,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,0,"POLYGON ((464544.04 4785291.952, 464544.04 478..."
1,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,1,"POLYGON ((464544.04 4786291.952, 464544.04 478..."
2,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,2,"POLYGON ((464544.04 4786291.952, 463774.88 478..."
3,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,3,"POLYGON ((463813.002 4787291.952, 463814.984 4..."
4,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,4,"POLYGON ((465544.04 4778291.952, 465544.04 477..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7713,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,7713,"POLYGON ((601544.04 4797291.952, 601544.04 479..."
7714,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,7714,"POLYGON ((601544.04 4798291.952, 601544.04 479..."
7715,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,7715,"POLYGON ((603086.973 4794291.952, 603054.567 4..."
7716,ESP.16_1,ESP,Spain,País Vasco,Basque Country|Baskenland|Basque,NA,Comunidad Autónoma,Autonomous Community,16,ES.PV,NA,7716,"POLYGON ((602544.04 4795291.952, 602728.754 47..."


Acá detectamos que tenemos una planta sin grid_id. (Estaba mal en mis datos, ya lo corregí, ahora se ve corregido):


In [ ]:
import matplotlib.pyplot as plt

# Dibuja los grids
grid.plot(edgecolor='black', facecolor='none', figsize=(10, 10))

# Superpone los puntos de las plantas
gdf_plantas.plot(color='red', markersize=20, ax=plt.gca())

# Resalta el punto problemático (EMPRESA 6)
#gdf_plantas[gdf_plantas["EMPRESA"] == 6].plot(color='blue', markersize=50, ax=plt.gca())
plt.show()

In [ ]:
print("CRS grid:", grid.crs)
print("CRS plantas:", gdf_plantas.crs)

CRS grid: EPSG:4326
CRS plantas: EPSG:4326


Ahora, agregamos a nuesto **df_consolidado** el campo ***tiene_planta***.

In [ ]:
# Extraer grid_ids únicos con al menos una planta
grids_con_planta = plantas_con_grid["grid_id"].dropna().unique()

# Marcar si un grid_id tiene una planta
df["tiene_planta"] = df["grid_id"].isin(grids_con_planta).astype(int)

In [ ]:
df[df['grid_id']==1235]

,grid_id,year,month,NO2,CO,SO2,O3,AAI,NTL,LST,PM25,tiene_planta
1235,1235,2018,6,0.000083,NaN,NaN,NaN,-0.522029,6.045000,15089.000000,6.640037e-09,1
8953,1235,2018,7,0.000081,0.026280,NaN,0.145077,-1.076895,7.120000,15182.750000,7.241857e-09,1
16671,1235,2018,8,0.000076,0.029617,NaN,0.138013,-0.554222,9.107500,15227.500000,7.631685e-09,1
24389,1235,2018,9,0.000087,0.027741,NaN,0.129446,-0.934655,8.982500,15149.500000,8.633419e-09,1
32107,1235,2018,10,0.000086,0.026849,0.000136,0.132300,-0.866884,6.555000,14659.000000,5.576595e-09,1
39825,1235,2018,11,0.000072,0.027048,NaN,0.135160,-0.707826,7.810000,14302.750000,3.959714e-09,1
47543,1235,2018,12,0.000063,0.030633,NaN,0.128426,-0.767409,8.130000,14089.250000,4.859054e-09,1
55261,1235,2019,1,0.000053,0.030344,NaN,0.135672,-0.705743,6.100000,14090.000000,4.382647e-09,1
62979,1235,2019,2,0.000068,0.030929,0.000815,0.136369,-1.041653,5.840000,14456.000000,9.000477e-09,1
70697,1235,2019,3,0.000080,0.034138,0.000305,0.145868,-1.116702,9.457500,14520.250000,5.830225e-09,1


##### Gráficos para los grids con planta

In [ ]:
# los grid id con planta son: 1235, 1825, 6618, 1265, 1119, 1190, 515, 1240
df_planta = df[df['grid_id']==1825]
df_planta['date'] = pd.to_datetime(df_planta[['year', 'month']].assign(day=1))
df_planta = df_planta.sort_values('date')

contaminantes = ['NO2', 'SO2', 'O3', 'CO', 'AAI', 'NTL', 'LST']

fig, axs = plt.subplots(len(contaminantes), 1, figsize=(15, 12), sharex=True)

for i, cont in enumerate(contaminantes):
    axs[i].bar(df_planta['date'], df_planta[cont], width=20)
    axs[i].set_ylabel(cont)
    axs[i].set_title(f'Concentración mensual de {cont}')
    axs[i].grid(True)

# Formatear eje x para mostrar año y mes
axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
axs[-1].xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # una etiqueta cada 3 meses
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## Agregamos nivel de producción a los grid con producción

In [ ]:
# Importamos produccion por empresa
df_produccion = pd.read_excel("./datos_empresa/produccion empresas vascas 18-22.xlsx", sheet_name="bd_produccion")

In [ ]:
# Asociamos cada empresa a su grid
empresa_grid = plantas_con_grid[['EMPRESA', 'grid_id']]
df_produccion = df_produccion.merge(empresa_grid, on = 'EMPRESA', how = 'left')

# Calculo de produccion por grid
produccion_grid = df_produccion.groupby(['grid_id','year','month'])['produccion'].sum().reset_index()

In [ ]:
# Agregamos la producción al df consolidad
df = df.merge(produccion_grid, on = ['grid_id','year','month'], how = 'left')

In [ ]:
df.to_csv("bd_consolidado.csv", index=False)

In [ ]:
df

,grid_id,year,month,NO2,CO,SO2,O3,AAI,NTL,LST,PM25,tiene_planta,produccion
0,0,2018,6,0.000072,NaN,NaN,NaN,-0.713915,1.2425,14872.50,6.237499e-09,0,NaN
1,1,2018,6,0.000072,NaN,NaN,NaN,-0.689671,1.1125,14840.50,6.237499e-09,0,NaN
2,2,2018,6,0.000072,NaN,NaN,NaN,-0.649017,0.4650,14833.25,6.237499e-09,0,NaN
3,3,2018,6,0.000071,NaN,NaN,NaN,-0.677410,0.4550,14829.25,6.237499e-09,0,NaN
4,4,2018,6,0.000070,NaN,NaN,NaN,-0.917463,0.4125,14811.00,5.899676e-09,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424485,7713,2022,12,0.000072,0.029296,NaN,0.131907,-0.206861,8.0000,14314.50,4.822519e-09,0,NaN
424486,7714,2022,12,0.000072,0.029296,NaN,0.131907,-0.206861,8.0000,14314.50,4.822519e-09,0,NaN
424487,7715,2022,12,0.000070,0.028963,NaN,0.131975,-0.201142,1.2325,14281.50,4.822519e-09,0,NaN
424488,7716,2022,12,0.000070,0.028963,NaN,0.131975,-0.201142,1.2325,14281.50,4.822519e-09,0,NaN


#### Transformación de PM25

In [ ]:
df = pd.read_csv("bd_consolidado.csv")

In [ ]:
df['PM25'] = df['PM25']*1000000

In [ ]:
df.to_csv("bd_consolidado.csv", index=False)

### Descarga de SO2 diario para dos meses
Se realiza esta descarga para validar que no tenemos datos para ningún día en meses en que todos los grids son missings

In [ ]:
def extract_daily_SO2(year, month, gdf_points, chunk_size=1000):
    from calendar import monthrange

    days_in_month = monthrange(year, month)[1]
    all_data = []

    for day in range(1, days_in_month + 1):
        date = datetime(year, month, day)
        next_day = date + timedelta(days=1)

        collection = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_SO2") \
            .filterDate(date.strftime('%Y-%m-%d'), next_day.strftime('%Y-%m-%d')) \
            .select("SO2_column_number_density")

        size = collection.size().getInfo()
        if size == 0:
            print(f"No hay datos para {date.strftime('%Y-%m-%d')}")
            continue

        image = collection.mean()

        for chunk in split_gdf(gdf_points, chunk_size):
            fc_chunk = geodf_to_fc(chunk)
            results = image.reduceRegions(
                collection=fc_chunk,
                reducer=ee.Reducer.mean(),
                scale=1000
            ).getInfo()

            for f in results["features"]:
                props = f["properties"]
                all_data.append({
                    "grid_id": props["grid_id"],
                    "SO2": props.get("mean", None),
                    "date": date.strftime('%Y-%m-%d')
                })

    return all_data

from datetime import datetime, timedelta

# Extraer datos diarios de SO2 para diciembre 2018
data_nov_2019 = extract_daily_SO2(2019, 11, points)

# Crear DataFrame
df_SO2_nov_2019 = pd.DataFrame(data_nov_2019)


In [ ]:
df_SO2_nov_2019.to_csv("SO2_diario_por_grid_nov2019.csv", index=False)

In [ ]:
def extract_daily_SO2(year, month, gdf_points, chunk_size=1000):
    from calendar import monthrange

    days_in_month = monthrange(year, month)[1]
    all_data = []

    for day in range(1, days_in_month + 1):
        date = datetime(year, month, day)
        next_day = date + timedelta(days=1)

        collection = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_SO2") \
            .filterDate(date.strftime('%Y-%m-%d'), next_day.strftime('%Y-%m-%d')) \
            .select("SO2_column_number_density")

        size = collection.size().getInfo()
        if size == 0:
            print(f"No hay datos para {date.strftime('%Y-%m-%d')}")
            continue

        image = collection.mean()

        for chunk in split_gdf(gdf_points, chunk_size):
            fc_chunk = geodf_to_fc(chunk)
            results = image.reduceRegions(
                collection=fc_chunk,
                reducer=ee.Reducer.mean(),
                scale=1000
            ).getInfo()

            for f in results["features"]:
                props = f["properties"]
                all_data.append({
                    "grid_id": props["grid_id"],
                    "SO2": props.get("mean", None),
                    "date": date.strftime('%Y-%m-%d')
                })

    return all_data

from datetime import datetime, timedelta

# Extraer datos diarios de SO2 para diciembre 2020
data_jun_2020 = extract_daily_SO2(2020, 6, points)

# Crear DataFrame
df_SO2_jun_2020 = pd.DataFrame(data_jun_2020)

In [ ]:
df_SO2_jun_2020.to_csv("SO2_diario_por_grid_jun2020.csv", index=False)

## Descarga de SO2 del proyecto OMI/AURA de Nasa
El producto OMI/Aura es una excelente alternativa para imputar valores faltantes de SO₂, especialmente en fechas donde Sentinel-5P no tiene cobertura. Aunque la resolución espacial de OMI (13×24 km) es más baja que la de Sentinel-5P (7×3.5 km o mejor), sigue siendo útil para llenar huecos o hacer análisis complementarios.
  
**Metodología**:  
- Se generó un archivo de texto directamente en la página https://search.earthdata.nasa.gov/ con los datos necesarios (filtros de fecha, zona y contaminante. Este archivo de texto se guarda. Contiene una dirección de descarga para cada día en la serie de tiempo filtrada.
- Se automatiza la descarga de cada archivo (son 2.866 archivos .he5).
- Los leemos en jupyter con una función con la que guardamos el nivel de SO2 para cada grid nuestro en una serie diaria. Para cruzar a nuestro grids la estrategia es: Cada polígono del pixel OMI se intersecta con todos nuestros grids (312 grids aprox tendran mismo valor) y para nuestros grids que se encuentran entre dos grids más grandes, se promedian los valores de los grids OMI.
- En un intento fallido noté muchos días con valores negativos y otros extremandamente negativos (-6e32 y superior) esta es una estrategia para marcar missings en la fuente. Por lo tanto, antes de realizar la agregación mensual:
    - Menores a 0 → 0.
    - menores a -1000 → NaN.
- Agregamos niveles de SO2 a nivel de grid con la media mensual.

In [ ]:
netrc_path = os.path.expanduser("~/.netrc")

with open(netrc_path, "w") as f:
    f.write("machine urs.earthdata.nasa.gov\n")
    f.write("login cvalladares\n")
    f.write("password 12011991Naci.\n")

os.chmod(netrc_path, 0o600)  # Seguridad del archivo

print(f"✅ Archivo .netrc creado en: {netrc_path}")

✅ Archivo .netrc creado en: C:\Users\camil/.netrc


In [ ]:
import requests
from getpass import getpass
import os

# Leer URLs
with open("OMSO2.txt", "r") as file:
    urls = [line.strip() for line in file if line.strip().endswith(".he5")]

# Credenciales
username = input("Usuario Earthdata (cvalladares): ")
password = getpass("Contraseña Earthdata(12011991Naci.): ")

# Carpeta de descarga
save_dir = "descargas_omi"
os.makedirs(save_dir, exist_ok=True)

# Sesión autenticada
session = requests.Session()
session.auth = (username, password)
session.headers.update({
    "User-Agent": "earthdata-downloader",
    "Accept": "*/*"
})

# IMPORTANTE: Login primero en el dominio de autenticación
login_url = "https://urs.earthdata.nasa.gov"
session.get(login_url)

# Descargar archivos
for url in urls:
    filename = os.path.join(save_dir, os.path.basename(url))
    if os.path.exists(filename):
        print(f"Ya existe: {filename}, omitiendo descarga.")
        continue
    print(f"Descargando: {filename}")
    with session.get(url, stream=True, allow_redirects=True) as r:
        if r.status_code == 200:
            with open(filename, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        else:
            print(f"❌ Error {r.status_code} al descargar: {url}")


Usuario Earthdata (cvalladares): cvalladares
Contraseña Earthdata(12011991Naci.): ········
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0101t1213-o71622_v003-2020m0222t183616.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0102t1117-o71636_v003-2020m0222t183151.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0102t1256-o71637_v003-2020m0222t183142.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0103t1200-o71651_v003-2020m0222t183544.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0104t1105-o71665_v003-2020m0222t183504.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0104t1244-o71666_v003-2020m0222t183457.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0105t1148-o71680_v003-2020m0222t183410.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0105t1327-o71681_v003-2020m0222t183410.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0106t1231-o71695_v003-2020m0222t184121.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0107t1136-o71709_v003-

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0301t1159-o72481_v003-2020m0222t194325.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0302t1104-o72495_v003-2020m0222t194305.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0302t1243-o72496_v003-2020m0222t194304.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0303t1147-o72510_v003-2020m0222t195115.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0303t1326-o72511_v003-2020m0222t195115.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0304t1230-o72525_v003-2020m0222t194918.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0305t1135-o72539_v003-2020m0222t194843.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0305t1314-o72540_v003-2020m0222t194839.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0306t1218-o72554_v003-2020m0222t195424.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0307t1123-o72568_v003-2020m0222t195401.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0307t1302-o72569_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0427t1338-o73312_v003-2020m0222t205250.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0428t1243-o73326_v003-2020m0222t205258.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0429t1147-o73340_v003-2020m0222t205309.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0429t1326-o73341_v003-2020m0222t205249.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0430t1231-o73355_v003-2020m0222t205302.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0501t1135-o73369_v003-2020m0222t205824.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0501t1314-o73370_v003-2020m0222t205824.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0502t1218-o73384_v003-2020m0222t205819.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0503t1123-o73398_v003-2020m0222t205754.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0503t1302-o73399_v003-2020m0222t205803.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0504t1206-o73413_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0619t2353-o74090_v003-2020m0222t221937.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0620t1125-o74097_v003-2020m0222t221926.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0620t1304-o74098_v003-2020m0222t221926.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0621t0036-o74105_v003-2020m0222t222136.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0621t1208-o74112_v003-2020m0222t221704.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0622t0119-o74120_v003-2020m0222t221651.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0622t1112-o74126_v003-2020m0222t221623.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0622t1251-o74127_v003-2020m0222t221611.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0623t0202-o74135_v003-2020m0222t222328.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0623t1156-o74141_v003-2020m0222t222257.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0623t1334-o74142_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0810t1153-o74840_v003-2020m0222t234115.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0810t1332-o74841_v003-2020m0222t234127.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0811t1236-o74855_v003-2020m0222t234408.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0812t0008-o74862_v003-2020m0222t235023.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0812t1141-o74869_v003-2020m0222t234712.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0812t1319-o74870_v003-2020m0222t234653.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0813t0052-o74877_v003-2020m0222t234650.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0813t1224-o74884_v003-2020m0222t234645.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0814t0135-o74892_v003-2020m0222t234700.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0814t1128-o74898_v003-2020m0222t234639.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2018m0814t1307-o74899_v00

...

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0504t1225-o94689_v003-2022m0505t030845.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0505t1129-o94703_v003-2022m0505t202619.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0505t1308-o94704_v003-2022m0506t025434.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0506t1212-o94718_v003-2022m0507t153306.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0507t1117-o94732_v003-2022m0507t212737.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0507t1256-o94733_v003-2022m0508t112948.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0508t1200-o94747_v003-2022m0509t054851.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0508t1339-o94748_v003-2022m0509t054025.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0509t1244-o94762_v003-2022m0510t042316.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0510t1148-o94776_v003-2022m0511t041450.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0510t1327-o94777_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0630t1220-o95519_v003-2022m0701t144058.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0701t1125-o95533_v003-2022m0705t142250.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0701t1304-o95534_v003-2022m0701t215656.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0702t1208-o95548_v003-2022m0705t142251.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0703t1112-o95562_v003-2022m0705t141747.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0703t1251-o95563_v003-2022m0703t221730.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0704t1156-o95577_v003-2022m0705t141236.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0704t1334-o95578_v003-2022m0705t142248.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0705t1239-o95592_v003-2022m0711t125107.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0706t1143-o95606_v003-2022m0711t125608.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0706t1322-o95607_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0828t1158-o96378_v003-2022m0829t035234.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0828t1337-o96379_v003-2022m0829t034736.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0829t1241-o96393_v003-2022m0830t024041.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0830t1145-o96407_v003-2022m0831t121540.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0830t1324-o96408_v003-2022m0831t121540.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0831t1229-o96422_v003-2022m0901t023939.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0901t1133-o96436_v003-2022m0902t024550.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0901t1312-o96437_v003-2022m0902t025124.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0902t1216-o96451_v003-2022m0904t121634.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0903t1120-o96465_v003-2022m0904t121641.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m0903t1259-o96466_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1024t1145-o97208_v003-2022m1025t120948.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1024t1324-o97209_v003-2022m1025t120945.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1025t1229-o97223_v003-2022m1026t042219.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1026t1133-o97237_v003-2022m1027t025359.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1026t1312-o97238_v003-2022m1027t025401.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1027t1216-o97252_v003-2022m1028t024638.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1028t1120-o97266_v003-2022m1028t205951.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1028t1259-o97267_v003-2022m1029t025421.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1029t1203-o97281_v003-2022m1030t115625.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1030t1108-o97295_v003-2022m1030t204835.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1030t1247-o97296_v00

Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1220t1136-o98038_v003-2022m1221t044438.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1220t1315-o98039_v003-2022m1221t044442.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1221t1219-o98053_v003-2022m1222t025757.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1222t1124-o98067_v003-2022m1222t205243.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1222t1303-o98068_v003-2022m1223t030824.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1223t1207-o98082_v003-2022m1224t170455.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1224t1111-o98096_v003-2022m1224t222738.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1224t1250-o98097_v003-2022m1225t130829.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1225t1155-o98111_v003-2022m1226t062112.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1226t1059-o98125_v003-2022m1226t211617.he5
Descargando: descargas_omi\OMI-Aura_L2-OMSO2_2022m1226t1238-o98126_v00

Con la celda anterior terminamos la descarga de archivos con los datos. A continuación los procesamos para crear nuestro data frame.

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
from shapely.geometry import Polygon
import re

# === 1. Cargar tus grids del País Vasco ===
gdf_grid = gpd.read_file("grilla_1km_pais_vasco.geojson")  # o .geojson, .gpkg
gdf_grid = gdf_grid.to_crs(epsg=4326)  # asegúrate de usar lat/lon
gdf_grid = gdf_grid[["grid_id", "geometry"]]  # solo columnas necesarias

# === 2. Carpeta de archivos OMI ===
folder = "descargas_omi"
files = sorted([f for f in os.listdir(folder) if f.endswith(".he5")])

In [ ]:
# === 3. Función para procesar un archivo ===
def procesar_archivo(fname, gdf_grid):
    try:
        path = os.path.join(folder, fname)
        with h5py.File(path, "r") as f:
            so2 = f["HDFEOS/SWATHS/OMI Total Column Amount SO2/Data Fields/ColumnAmountSO2"][:]
            lat = f["HDFEOS/SWATHS/OMI Total Column Amount SO2/Geolocation Fields/Latitude"][:]
            lon = f["HDFEOS/SWATHS/OMI Total Column Amount SO2/Geolocation Fields/Longitude"][:]

        rows, cols = so2.shape
        records = []

        for i in range(rows - 1):
            for j in range(cols - 1):
                # Coordenadas del píxel como polígono
                latitudes = [lat[i, j], lat[i, j+1], lat[i+1, j+1], lat[i+1, j]]
                longitudes = [lon[i, j], lon[i, j+1], lon[i+1, j+1], lon[i+1, j]]

                if np.any(np.isnan(latitudes)) or np.any(np.isnan(longitudes)):
                    continue

                polygon = Polygon(zip(longitudes, latitudes))
                valor_so2 = so2[i, j]

                # Reglas para valores
                if valor_so2 < -1000:
                    valor_so2 = np.nan
                elif valor_so2 < 0:
                    valor_so2 = 0

                records.append({"geometry": polygon, "SO2": valor_so2})

        if not records:
            return None

        gdf_omi = gpd.GeoDataFrame(records, crs="EPSG:4326")

        # Cruce espacial por intersección (más preciso que "within")
        gdf_joined = gpd.sjoin(gdf_grid, gdf_omi, how="inner", predicate="intersects")

        # Extraer fecha
        match = re.search(r"(\d{4})m(\d{2})\d{2}t", fname)
        if match:
            anio = int(match.group(1))
            mes = int(match.group(2))
        else:
            raise ValueError(f"No se pudo extraer la fecha del archivo: {fname}")

        gdf_joined["year"] = anio
        gdf_joined["month"] = mes

        return gdf_joined[["grid_id", "SO2", "year", "month"]]

    except Exception as e:
        print(f"❌ Error procesando {fname}: {e}")
        return None


In [ ]:
# === 4. Procesar todos los archivos ===
resultados = []

for fname in tqdm(files):
    resultado = procesar_archivo(fname, gdf_grid)
    if resultado is not None and not resultado.empty:
        resultados.append(resultado)

# === 5. Unir y agregar por grid/mes ===
if resultados:
    df_all = pd.concat(resultados)
    df_monthly = df_all.groupby(["grid_id", "year", "month"], as_index=False).mean()
    df_monthly.to_csv("SO2_OMI_por_grid_mensual.csv", index=False)
    print("✅ Archivo guardado: SO2_OMI_por_grid_mensual.csv")
else:
    print("❌ No se extrajo ningún dato válido.")

100%|████████████████████████████████████████████████████████████████████████████| 2866/2866 [3:37:33<00:00,  4.55s/it]


✅ Archivo guardado: SO2_OMI_por_grid_mensual.csv


In [ ]:
# Filtra los que no son None (por si hubo errores)
resultados_validos = [r for r in resultados if r is not None]

# Unir en un único DataFrame
df_omi = pd.concat(resultados_validos, ignore_index=True)